In [1]:
%load_ext Cython
import sys
sys.path[0] = ('/home/bram/ANTS/')


In [2]:
%%cython
# from cythonic.core.domain cimport domain

from cythonic.plugins.positions cimport point, index
from cythonic.core.map cimport MeshMap, GaussMap
cimport numpy as np
import numpy as np
from libc.math cimport ceil as cceil, sqrt as csqrt, log as cln

cdef class domain:
    cdef:
        readonly point size, nest_location, food_location
        readonly double pitch, nest_radius, food_radius
        readonly MeshMap Map
        readonly GaussMap Gaussian
        readonly index dim
    def __cinit__(self,size,pitch,nest_loc, nest_rad, food_loc, food_rad):
        self.size = point(*size)
        self.nest_location = point(*nest_loc)
        self.food_location = point(*food_loc)
        self.nest_radius = nest_rad
        self.food_radius = food_rad

        self.Map = MeshMap(dim = np.array(size, dtype=np.float_), resolution = <double>pitch)
        self.dim = index(*np.array(self.Map.map).shape)


cdef double pitch = .1
dom = domain(size = [100,100],pitch= pitch, food_loc = [25,25], food_rad = 10, nest_loc = [75,75], nest_rad = 10)
cdef double i = 1.1
print(cceil(i))
print(csqrt(2.))
print(cln(2))
print(np.log(2))

cdef double significancy = 1e3, sigma = 10

print(np.asarray(np.ceil(sigma*np.sqrt(2*np.log(significancy))),dtype=int))
print( cceil(sigma*csqrt(2*cln(significancy))))

2.0
1.4142135623730951
0.6931471805599453
0.6931471805599453
38
38.0


In [3]:
%%cython
from cythonic.core.domain cimport domain
import numpy as np
from cythonic.plugins.positions cimport point
cdef double pitch = .1
cdef domain dom = domain(size = [100,100],pitch= pitch, food_loc = [25,25], food_rad = 10, nest_loc = [75,75], nest_rad = 10)
dom.init_gaussian(1.,10.)
cdef double x = dom.probe_pheromone(point(10.,10.))
print(x)
print("Volume of the map == {}".format(np.array(dom.Gaussian.map).sum()*pitch**2))

1.0
Volume of the map == 0.995447696073483


In [24]:
%%cython -a
import cython
from cythonic.core.domain cimport domain
from cythonic.plugins.positions cimport map_range, index, point
import numpy as np
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef class Domain(domain):
    cdef readonly void add_pheromone(self,double[:,::1] map, double[:,::1] gauss, point p, double Q):
        " add quantity Q pheromone as gaussian centered around p "
        cdef index I = index(self.Map.to_grid(&p.xy[0]),self.Map.to_grid(&p.xy[1]))
        cdef map_range s = self.Map.span(I.xy,&self.Gaussian.radius)
        cdef long offset_x = self.Map.to_grid(&p.xy[0])-s.x[1]+s.x[0]
        cdef long offset_y = self.Map.to_grid(&p.xy[1])-s.y[1]+s.y[0]
        cdef long i,j
        for i in range(s.y[2]-s.y[0]):
            for j in range(s.x[2]-s.x[0]):
                map[offset_y+i,offset_x+j]+=Q*gauss[i+s.y[0], j+s.x[0]]
        
cdef double pitch = 1     
cdef Domain dom = Domain(size = [10,7],pitch= pitch, food_loc = [25,25], food_rad = 10, nest_loc = [75,75], nest_rad = 10)
dom.init_gaussian(.5,1e3)
print(np.array(dom.Map.map).shape)
# dom.add_pheromone(point(0,0), Q = 5.)
print(np.array(dom.Map.map))
print(np.array(dom.Map.mesh_x))
print(np.array(dom.Map.mesh_y))
print(dom.Map.mesh_y[7,1])

from time import time
cdef double tic, toc, Q = 3.
cdef long i,N = 1000
cdef point p = point(5,5)
tic = time()
for i in range(N):
    dom.add_pheromone(dom.Map.map,dom.Gaussian.map,p,Q)
toc = time()
print(f"avg {1000*(toc-tic)/N} msec")
print(np.array(dom.Map.map).max(), N*np.array(dom.Gaussian.map).max())

(8, 11)
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]]
7.0
avg 0.0007212162017822266 msec
19

In [1]:
%%cython -a
from cython cimport wraparound, boundscheck
from cythonic.core.domain cimport domain
from cythonic.plugins.positions cimport map_range, index, point
import numpy as np
cimport numpy as np

cdef double pitch = 1

cdef class Domain(domain):
    def pyadd_pheromone(self,point p, double Q):
        """ Add pheromone to the temp map:
                Q is the quantity
                target_pos x,y in mm """
        mp = np.array(self.Map.map)
        gauss = np.array(self.Gaussian.map)
        cdef index I = index(self.Map.to_grid(&p.xy[0]),self.Map.to_grid(&p.xy[1]))
        cdef map_range s = self.Map.span(I.xy,&self.Gaussian.radius)
        cdef long offset_x = self.Map.to_grid(&p.xy[0])-s.x[1]+s.x[0]
        cdef long offset_y = self.Map.to_grid(&p.xy[1])-s.y[1]+s.y[0]
#         map[offset_y:offset_y+s.y[2]-s.y[0],offset_x:offset_x+s.x[2]-s.x[0]]+= \
#             Q*gauss[s.y[0]:s.y[2],s.x[0]:s.x[2]]

         
cdef Domain dom = Domain(size = [10,7],pitch= pitch, food_loc = [25,25], food_rad = 10, nest_loc = [75,75], nest_rad = 10)

cdef double tic, toc, Q = 3.
cdef point p = point(5,5)

dom.pyadd_pheromone(p,Q)

cdef long i,N = 1000

UsageError: Cell magic `%%cython` not found.


In [48]:
%%cython -a
cimport numpy as np
import numpy as np
cdef double[:,::1] mv = np.ones((10,5))

cdef double[:,::1] mv2 = np.asaray(<np.int32_t[:10, :10]> mv)




Error compiling Cython file:
------------------------------------------------------------
...
cimport numpy as np
import numpy as np
cdef double[:,::1] mv = np.ones((10,5))

cdef double[:,::1] mv2 = np.asaray(<np.int32_t[:10, :10]> mv)
                                                         ^
------------------------------------------------------------

/home/bram/.cache/ipython/cython/_cython_magic_12f8d09bb4b079cbd01883b6aa9b95fb.pyx:5:58: Can only create cython.array from pointer or array


TypeError: object of type 'NoneType' has no len()

In [73]:
%%cython -a
cimport cython
cimport numpy as np
import numpy as np
from cythonic.plugins.positions cimport map_range
from libc.math cimport lrint # round double, cast as long
from libc.math cimport ceil as cceil, sqrt as csqrt, log as cln
cdef struct index:
    unsigned long x
    unsigned long y

cdef struct point:
    double x
    double y

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef class Map:
    """ base class for a meshgrid and some supporting functions """
    "map == 2D array map[x,y]"
    cdef:
        readonly double pitch
        readonly point dim
        readonly double[:,::1] map, mesh_x, mesh_y
        readonly index lim

    cdef readonly map_range span(self,unsigned long *Ix, unsigned long *Iy,double *R) nogil:
        """ return the upper limits of a meshmap that fits in self.map
            I as index pair xy -> grid coords[uint, uint],
            R as offset in mm """
        cdef map_range XY # store result
        cdef unsigned long R_int = self.to_grid(R)
        XY.x[1] = R_int
        XY.y[1] = R_int
        if Ix[0] +R_int+1 > self.lim.x: # ubound(x)
            XY.x[2] = R_int-Ix[0]+self.lim.x+1
        else: XY.x[2] = 2*R_int+1
        if Iy[0] +R_int+1 > self.lim.y: #ubound(y)
            XY.y[2] = R_int -Iy[0]+self.lim.y+1
        else: XY.y[2] = 2*R_int+1
        if Ix[0] < R_int: XY.x[0] = R_int - Ix[0] #lbound(x)
        else: XY.x[0] = 0
        if Iy[0] < R_int: XY.y[0] = R_int - Iy[0] #lbound(y)
        else: XY.y[0] = 0
        return XY

    cdef readonly unsigned long to_grid(self, double * x) nogil:
        " round point in mm to grid index "
        return lrint(x[0]/self.pitch)
    cdef readonly double to_mm(self,unsigned long * x) nogil:
        " convert grid index to location in mm "
        return <double>self.pitch*x[0]

    def __repr__(self):
        " print useful information "
        return "Map with size ({}x{}), pitch = {};".format(
            self.dim.x, self.dim.y,self.resolution)

    def new(self, double[:] dim, double resolution):
        " assume dim is vector[2]=> [double x, double y]"
        self.dim = point(dim[0], dim[1])
        self.pitch = resolution
        self.lim = index(self.to_grid(&self.dim.x),self.to_grid(&self.dim.y))

        self.mesh_x, self.mesh_y = np.meshgrid(\
                np.arange(0,(<double>self.lim.x)*resolution+resolution, resolution),
                np.arange(0,(<double>self.lim.y)*resolution+resolution, resolution))
        self.map = np.ones((self.lim.y+1,self.lim.x+1),dtype = np.float_)

cdef class MeshMap(Map):
    " Map wrapper "
    def __cinit__(self,double[:] dim, double resolution):
        self.new(dim,resolution)
        
cdef class GaussMap(Map):
    cdef double radius, volume, peak
    def __cinit__(self,double resolution, double R, double covariance):
        self.radius = R
        dim = np.array([self.radius*2,self.radius*2], dtype = np.float_)
        self.new(dim,resolution)
        self.map = self.bivariate_normal(sigmax=covariance,sigmay=covariance,
            mux = self.dim.x/2, muy=self.dim.y/2, sigmaxy=0)
        self.volume = np.array(self.map).sum()*self.pitch**2
        self.peak = np.array(self.map).max()

        # print(f"volume = {self.volume}\n peak at {self.peak}")

    def bivariate_normal(self,sigmax=1.0, sigmay=1.0,
                         mux=0.0, muy=0.0, sigmaxy=0.0):
        """
        X (meshgrid),Y (meshgrid), sigmax (scalar), sigmay (scalar), mux (scalar),
        muy (scalar) , sigmaxy(scalar)
        FROM https://github.com/matplotlib/matplotlib/blob/81e8154dbba54ac1607b21b22984cabf7a6598fa/lib/matplotlib/mlab.py#L1866
        Bivariate Gaussian distribution for equal shape *X*, *Y*.
        See `bivariate normal
        <http://mathworld.wolfram.com/BivariateNormalDistribution.html>`_
        at mathworld.
        """
        X = np.array(self.mesh_x)
        Y = np.array(self.mesh_y)
        Xmu = X-mux
        Ymu = Y-muy

        rho = sigmaxy/(sigmax*sigmay)
        z = Xmu**2/sigmax**2 + Ymu**2/sigmay**2 - 2*rho*Xmu*Ymu/(sigmax*sigmay)
        denom = 2*np.pi*sigmax*sigmay*np.sqrt(1-rho**2)
        return np.exp(-z/(2*(1-rho**2))) / denom
        
@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef class domain:
    " playground of the simulation "
    cdef:
        readonly point size, nest_location, food_location
        readonly double pitch, nest_radius, food_radius
        readonly MeshMap Map
        readonly GaussMap Gaussian
        readonly index dim
        public double target_pheromone
        
    cdef readonly void init_gaussian(self, double sigma, double significancy):
        " initialize a gaussian meshmap "
        cdef double R = cceil(sigma*csqrt(2*cln(significancy)))
        self.Gaussian = GaussMap(resolution = self.Map.pitch, R = R, covariance = sigma)

    cdef bint check_bounds(self, double* x, double* y) nogil:
        " check if point is within the domain limits"
        if x[0] >=0 and y[0]>=0 and \
            x[0]<=self.size.x and y[0]<= self.size.y:
            return True
        else:
            return False

    cdef readonly double probe_pheromone(self,point p, double[:,::1] mv):
        " return the pheromone quantity at a location on the map "
        cdef double Q
        if self.check_bounds(&p.x, &p.y):
            " convert point to index "
            Q = mv[self.Map.to_grid(&p.y),self.Map.to_grid(&p.x)]
        else:
            " out of bounds, return zero "
            Q = 0.
        return Q
    
#     cdef readonly void add_pheromone(self,double[:,::1] mp, double[:,::1] gauss, point *p, double *Q):
    cdef readonly void add_pheromone(self,point *p, double *Q):
        " add quantity Q pheromone at gaussian centered around p "
        cdef index I = index(self.Map.to_grid(&p.x),self.Map.to_grid(&p.y))
        cdef map_range s = self.Map.span(&I.x,&I.y,&self.Gaussian.radius)
        cdef long offset_x = self.Map.to_grid(&p.x)-s.x[1]+s.x[0]
        cdef long offset_y = self.Map.to_grid(&p.y)-s.y[1]+s.y[0]
        cdef long i,j
        with nogil:
            for i in range(s.y[2]-s.y[0]):
                for j in range(s.x[2]-s.x[0]):
                    self.Map.map[offset_y+i,offset_x+j]+=Q[0]*self.Gaussian.map[i+s.y[0], j+s.x[0]]
#                 mp[offset_y+i,offset_x+j]+=Q[0]*gauss[i+s.y[0], j+s.x[0]]
        



    def __cinit__(self,size,pitch,nest_loc, nest_rad, food_loc, food_rad):
        self.size = point(size[0],size[1])
        self.nest_location = point(nest_loc[0],nest_loc[1])
        self.food_location = point(food_loc[0],food_loc[1])
        self.nest_radius = nest_rad
        self.food_radius = food_rad

        self.Map = MeshMap(dim = np.array(size, dtype = np.float_), resolution = pitch)
        self.dim = index(self.Map.map.shape[0],self.Map.map.shape[1])

cdef double pitch = .1
cdef domain dom = domain(size = [10,7],pitch= pitch, food_loc = [25,25], food_rad = 10, nest_loc = [75,75], nest_rad = 10)
dom.init_gaussian(.5,1e3)
print(np.array(dom.Map.map).shape)
# dom.add_pheromone(point(0,0), Q = 5.)
print(np.array(dom.Map.map))
print(np.array(dom.Map.mesh_x))
print(np.array(dom.Map.mesh_y))
print(dom.Map.mesh_y[7,1])

from time import time
cdef double tic, toc, Q = 3.
cdef long i,N = 10000
cdef point p = point(5,5)
tic = time()
with cython.boundscheck(False), cython.nonecheck(False),cython.cdivision(True),cython.wraparound(False):
    for i in range(N):
        dom.add_pheromone(&p,&Q)
#         dom.add_pheromone(dom.Map.map,dom.Gaussian.map,&p,&Q)
toc = time()
print(f"avg {1000*(toc-tic)/N} msec")
print(np.array(dom.Map.map).max(), N*np.array(dom.Gaussian.map).max())
        

(71, 101)
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[[ 0.   0.1  0.2 ...  9.8  9.9 10. ]
 [ 0.   0.1  0.2 ...  9.8  9.9 10. ]
 [ 0.   0.1  0.2 ...  9.8  9.9 10. ]
 ...
 [ 0.   0.1  0.2 ...  9.8  9.9 10. ]
 [ 0.   0.1  0.2 ...  9.8  9.9 10. ]
 [ 0.   0.1  0.2 ...  9.8  9.9 10. ]]
[[0.  0.  0.  ... 0.  0.  0. ]
 [0.1 0.1 0.1 ... 0.1 0.1 0.1]
 [0.2 0.2 0.2 ... 0.2 0.2 0.2]
 ...
 [6.8 6.8 6.8 ... 6.8 6.8 6.8]
 [6.9 6.9 6.9 ... 6.9 6.9 6.9]
 [7.  7.  7.  ... 7.  7.  7. ]]
0.7000000000000001
avg 0.0013994693756103516 msec
19099.59317102644 6366.197723675814
